1. A user mentions bot @parrepartition
2. The script finds that new @parrepartition mention and then reads the parent tweet it is being mentioned on
3. The script takes that parent tweet and generates a response relevant with the French pension system using a language model
4. Respond is posted and tweet is logged

This notebook focuses on #3

In [1]:
pip install langchain

  Using cached langchain-0.2.1-py3-none-any.whl.metadata (13 kB)
  Using cached SQLAlchemy-2.0.30-cp312-cp312-macosx_10_9_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_10_9_x86_64.whl.metadata (7.5 kB)
  Using cached langchain_text_splitters-0.2.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 863.6 kB/s eta 0:00:001m965.5 kB/s eta 0:00:01
  Using cached tenacity-8.3.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-macosx_10_9_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-macosx_10_9_x86_64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (31 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata

In [3]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
llm = ChatOpenAI(temperature=0.7,
                 api_key=OPENAI_API_KEY,
                 model_name='gpt-3.5-turbo',
                )

Then let's create our function that will take in a piece of a text (a tweet) and give us an output response

    You are a French economist and a professor of finance at Wharton School of the University of Pennsylvania. You are critical but unbiased about the French unfunded pension system. You think that introducing a dose of private and funded pension plans into the French system would make it more sustainable and beneficial to French pensioners. Your goal is to give a concise prediction in response to a piece of text from a French user.

    % RESPONSE TONE:

        Your prediction should be opinionated, factual and given in a respectable voice
        Your tone should be serious but not too much

    % RESPONSE FORMAT:

        Respond in French
        Respond in under 200 characters
        Respond in one short sentence
        Do not respond with emojis

    % RESPONSE CONTENT:

        Include specific examples of pension systems if they are relevant
        If you don't have an answer, say, "Je ne sais pas quoi penser de votre avis."
